In [ ]:
import torch
import torch.nn as nn


### How does VAE Works?
1) The output of the encoder is two vectors:
- Mean vector
- Standard Deviation

2) We sum the two vectors(mean and variation) which is mutliplied by small random number. The dimensions of the resultant vector will same to that of each vectors.

3) The resultant vector is passed to the decoder to fetch the images.

4) Loss value optimization is combination of:
- KL Divergence loss: measures the deviation of distribution of mean vector and standard variance vector from 0 and 1 resp.
-  MSE: optimization we use to reconstruct the image.

In [ ]:
class VAE(nn.Module):
  def __init__(self, x_dim, h_dim1, h_dim2, latent_dim):
    super(VAE, self).__init__()
    #encoder section:
    self.d1=nn.Linear(x_dim, h_dim1) #input -> hidden layer 1
    self.d2=nn.Linear(h_dim1, h_dim2) #hidden layer 1 -> hidden layer 2

    # mean and standard variation vectors:
    self.d31=nn.Linear(h_dim2, latent_dim) # hidden layer 2 -> Mean
    self.d32=nn.Linear(h_dim2, latent_dim) # hidden layer 2 -> log variance

    #decoder section:
    self.d4=nn.Linear(latent_dim, h_dim2) # latent -> hidden layer 2
    self.d5=nn.Linear(h_dim2, h_dim1) #hidden layer 2 -> hidden layer 1
    self.d6=nn.Linear(h_dim1, x_dim)